### Copyright 2024 Lehi Gracia

### Markets

In [1]:

from yfinance_utils import list_utils, file_utils, constants
from yfinance import Ticker
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA

symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

gains = {}
for symbol in symbols:
    t = Ticker(symbol)
    df1 = t.history(period="1y")
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



/Users/nachos/Documents/GitHub/yfinance_utils/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


PERFORMANCE IN THE LAST 30 DAYS:
SPY : -0.82: SNP 500
DIA : -3.75: DOW JONES
QQQ :  2.02: NASDAQ 100


### Sectors

In [2]:

symbols = list_utils.get_sectors()
names = list_utils.sectors

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="SECTORS", title_x=0.5, width=800, height=400)

gains = {}
for symbol in symbols:
    t = Ticker(symbol)
    df1 = t.history(period='1y')
    df1 = df1[-30:]
    df1.reset_index(inplace=True)
    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:6}:{gains[names[symbol]]:8}: {names[symbol]}')


PERFORMANCE IN THE LAST 30 DAYS:
XLP   :    -4.9: CONSUMER STAPLES
XLK   :    0.32: TECHNOLOGY
XLU   :   -5.88: UTILITIES
XLY   :    2.16: CONSUMER DISCRETIONARY
XLB   :  -10.58: MATERIALS
XLI   :   -6.79: INDUSTRIALS
XLV   :   -2.65: HEALTH CARE
XLF   :   -4.32: FINANCIALS
XLRE  :   -8.35: REAL ESTATE
XLE   :   -8.83: ENERGY
XLC   :    1.25: COMMUNICATION SERVICES


### Ratings from at least 50 analysts, higher than 85% UP

In [3]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)


client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'TOTAL':{"$gt":40}, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 677e87506b9c1a87197401c8, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

### Gap up close high

In [108]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)

          DATE  TICK     HIGH    CLOSE     OPEN  CLOSE -1
0   2025-01-07  MPWR   644.81   643.07   633.10    632.37
1   2025-01-07   ADI   220.27   219.79   217.47    215.33
2   2025-01-07   CNC    63.18    62.88    62.77     61.76
3   2025-01-07   WMT    91.94    91.70    91.06     90.83
4   2025-01-07  HUBB   441.07   440.55   437.77    433.76
5   2025-01-07    BK    78.49    78.13    78.08     77.78
6   2025-01-07   WFC    73.12    72.97    72.03     71.93
7   2025-01-07    IT   493.61   493.33   492.80    487.11
8   2025-01-07   BBY    85.57    85.18    84.64     84.06
9   2025-01-07   DGX   154.00   153.28   153.17    152.46
10  2025-01-07  META   632.10   631.49   619.32    611.83
11  2025-01-07   HPE    23.03    23.05    22.45     22.22
12  2025-01-07  DELL   125.77   125.58   122.16    121.81
13  2025-01-07  PYPL    89.97    89.55    88.30     88.02
14  2025-01-07   PEG    86.55    86.33    85.59     85.56
15  2025-01-07  ASML   778.39   774.99   762.07    751.06
16  2025-01-07

### Price drops and jumps

In [109]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
print(data)


         DATE  TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
0  2025-01-07  QBTS   10.44         13.85       -41.96        43.43
1  2025-01-07   SMR   23.59          6.02       -36.14        45.23
2  2025-01-07  IONQ   51.28          6.17       -13.48       -17.48
3  2025-01-07    MU  103.40          8.19        -5.25       145.38
4  2025-01-07   NNE   32.33         15.30       -52.42        95.43
5  2025-01-07  INCY   73.04          5.17       -45.89        25.22
6  2025-01-07  OKLO   30.00          9.25       -50.45        12.47
7  2025-01-07  LULU  404.42          6.03       -29.79        67.53
8  2025-01-07  AXON  570.72         -5.57       -65.78       271.49
9  2025-01-07  SERV   19.49         -6.84       -25.82        36.30
